In [27]:
import findspark
findspark.init()

In [28]:
# Time analysis for pandas read and write operations

import pandas as pd
import time
from pyspark.sql import SparkSession

In [29]:
spark = SparkSession.builder \
    .appName("My Spark Application") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.13.7") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "6g") \
    .config("spark.executor.memoryOverhead", "1g") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.sql.debug.maxToStringFields", 1000) \
    .getOrCreate()

# analysis with pandas

In [30]:
# Start the timer
start_time = time.time()

# Read the Excel file
df_layoff_pd = pd.read_excel('tech_layoffs.xlsx')

# Read the TSV file
df_housing_pd = pd.read_csv("latest_weekly_housing_market_data_most_recent.tsv000", sep='\t')

# Stop the timer
end_time = time.time()

# Calculate total duration
total_duration = end_time - start_time

print(f"Total time to run the two lines of code: {total_duration} seconds")

Total time to run the two lines of code: 61.61105275154114 seconds


# analysis with Pyspark

In [31]:
# Start timer for both operations
start_time = time.time()

# Read Excel file
df_layoff_sk = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("dataAddress", "'Sheet1'!A1") \
    .load('tech_layoffs.xlsx')

# Read CSV file
df_housing_sk = spark.read.csv('latest_weekly_housing_market_data_most_recent.tsv000', header=True, sep='\t')

# Calculate and print the total time taken for both read operations
total_time_taken = time.time() - start_time
print(f"Total time taken to read both files: {total_time_taken} seconds")

Total time taken to read both files: 0.7997944355010986 seconds


# Parquet write and read with pandas

In [32]:
import time

# Start the timer
start_time = time.time()

# Write the DataFrame from the Excel file to a Parquet file
df_layoff_pd.to_parquet('tech_layoffs_pd.parquet')

# Write the DataFrame from the TSV file to a Parquet file
df_housing_pd.to_parquet('housing_market_pd.parquet')

# Stop the timer
end_time = time.time()

# Calculate total duration
total_duration = end_time - start_time

print(f"Total time to write both DataFrames to Parquet files: {total_duration} seconds")

Total time to write both DataFrames to Parquet files: 18.741501569747925 seconds


# parallel write to parquet with pyspark

In [33]:
#TODO: experimentation write parallel and more memory

In [34]:
# Start the timer
start_time = time.time()

# Read the Parquet file into a DataFrame
df_layoff_pd_pq = pd.read_parquet('tech_layoffs_pd.parquet')
df_housing_pd_pq = pd.read_parquet('housing_market_pd.parquet')

# Stop the timer
end_time = time.time()

# Calculate total duration
total_duration = end_time - start_time

print(f"Total time to read both Parquet files into DataFrames: {total_duration} seconds")

Total time to read both Parquet files into DataFrames: 4.858309984207153 seconds


# Parquet write and read with pyspark

In [36]:
# Start timer for both write operations
start_time = time.time()

# Write the DataFrame to a Parquet file
df_layoff_sk.write.parquet('tech_layoffs_sk.parquet', mode='overwrite')

# Write the DataFrame to a Parquet file
df_housing_sk.write.parquet('housing_market_sk.parquet', mode='overwrite')

# Calculate and print the total time taken for both write operations
total_time_taken = time.time() - start_time
print(f"Total time taken to write both DataFrames to Parquet files: {total_time_taken} seconds")


Py4JJavaError: An error occurred while calling o128.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(Fi

In [ ]:
# Start timer for both read operations
start_time = time.time()

# Read the Parquet file into a DataFrame
df_layoff_sk_pq = spark.read.parquet('tech_layoffs_sk.parquet')

# Read the Parquet file into a DataFrame
df_housing_sk_pq = spark.read.parquet('housing_market_sk.parquet')

# Calculate and print the total time taken for both read operations
total_time_taken = time.time() - start_time
print(f"Total time taken to read both DataFrames from Parquet files: {total_time_taken} seconds")

Py4JJavaError: An error occurred while calling o106.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:180)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anon